In [1]:
import random
import math
import os
import pickle
from tqdm import tqdm, trange
import numpy as np

In [2]:
def pickler(path,pkl_name,obj):
    with open(os.path.join(path, pkl_name), 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def unpickler(path,pkl_name):
    with open(os.path.join(path, pkl_name) ,'rb') as f:
        obj = pickle.load(f)
    return obj

In [3]:
out_pkl_path = "./"
in_pkl_path = "./"
in_pkl_name = "preprocessed_train.pkl"
out_pkl_name = "preprocessed_train_balanced.pkl"

In [4]:
original_data = unpickler(in_pkl_path,in_pkl_name)

In [5]:
def segregare_data(data):
    positive = {"sequences":[], "segment_ids":[], "supporting_fact":[]}
    negative = {"sequences":[], "segment_ids":[], "supporting_fact":[]}
    for i in trange(len(data["supporting_fact"])):
        if(data["supporting_fact"][i] == 1):
            for key in positive.keys():
                positive[key].append(data[key][i])
        else:
            for key in negative.keys():
                negative[key].append(data[key][i])
    return positive, negative

In [6]:
positive, negative = segregare_data(original_data)

100%|██████████| 3706225/3706225 [00:05<00:00, 631883.63it/s]


In [7]:
print(positive["sequences"][0])
print(positive["segment_ids"][0])
print(positive["supporting_fact"][0])

[101, 2073, 2001, 1996, 2364, 2839, 1999, 1996, 2143, 22953, 15551, 14620, 1999, 2152, 1011, 3036, 13691, 8323, 1029, 102, 2002, 2038, 2985, 6993, 14620, 1999, 1996, 13276, 2669, 1010, 5041, 17622, 1998, 6683, 5172, 2152, 1011, 3036, 13691, 8323, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
1


In [8]:
print(negative["sequences"][0])
print(negative["segment_ids"][0])
print(negative["supporting_fact"][0])

[101, 2073, 2001, 1996, 2364, 2839, 1999, 1996, 2143, 22953, 15551, 14620, 1999, 2152, 1011, 3036, 13691, 8323, 1029, 102, 2798, 4300, 1000, 4918, 1000, 10582, 1006, 2141, 2745, 5146, 12001, 2036, 2124, 2004, 2798, 22953, 15551, 1025, 1020, 2285, 3999, 1007, 2003, 2019, 2394, 4735, 2040, 2003, 2411, 3615, 2000, 1999, 1996, 2329, 2811, 2004, 1996, 1000, 2087, 6355, 7267, 1999, 3725, 1000, 1998, 1000, 3725, 1005, 1055, 2087, 12536, 7267, 1000, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
0


In [9]:
print(min(negative["supporting_fact"]))
print(max(negative["supporting_fact"]))

0
0


In [10]:
print(min(positive["supporting_fact"]))
print(max(positive["supporting_fact"]))

1
1


In [11]:
num_positive = len(positive["supporting_fact"])
num_negative = len(negative["supporting_fact"])

In [12]:
print(num_positive)
print(num_negative)

215932
3490293


In [13]:
def get_random_subset(data, output_size):
    indices = random.sample(list(range(len(data["supporting_fact"]))), output_size)
    out_dict = {"sequences":[], "segment_ids":[], "supporting_fact":[]}
    for i in indices:
        for key in out_dict.keys():
            out_dict[key].append(data[key][i])
    return out_dict

In [14]:
negative_sample_size_multiplier = 3

In [15]:
negative_sample_size = num_positive * negative_sample_size_multiplier

In [16]:
negative_subset = get_random_subset(negative, negative_sample_size)

In [17]:
print(len(negative_subset["supporting_fact"]))

647796


In [18]:
def merge(dict_list):
    out_dict = {"sequences":[], "segment_ids":[], "supporting_fact":[]}
    for key in out_dict.keys():
        for d in dict_list:
            out_dict[key] += d[key]
    return out_dict

In [19]:
positive_repeated = []
for i in range(negative_sample_size_multiplier):
    positive_repeated.append(positive)
    

out_dict = merge(positive_repeated + [negative_subset])

In [20]:
print(len(out_dict["supporting_fact"]))

1295592


In [21]:
print(min(out_dict["supporting_fact"]))
print(max(out_dict["supporting_fact"]))

0
1


In [22]:
sum(out_dict["supporting_fact"])

647796

In [23]:
pickler(out_pkl_path, out_pkl_name, out_dict)
print("Done")

Done
